In [2]:
import os
import pandas as pd
import arcpy
import arcpy.mp as mp
import re
import requests
import numpy as np
import geopandas

#### Get the path of this notebook and use it to generate path for input shapefile:

In [3]:
current_dir = os.getcwd()
shapefile_cbg = r"input\cbg_kontur.shp"
shapefile_cbg_path = os.path.join(current_dir, shapefile_cbg)
output_folder_path = os.path.join(current_dir, "output")
print(f"Location of this notebook: {current_dir}")
print(f"Location of CBG SHP: {shapefile_cbg_path}")
print(f"Location for output: {output_folder_path}")

Location of this notebook: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis
Location of CBG SHP: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\input\cbg_kontur.shp
Location for output: C:\GITHUB\CCSVI\Scripts\Spatial_Analysis\output


#### Set up ArcPy Workspace:

In [4]:
arcpy.env.overwriteOutput = True
workspace = os.path.dirname(shapefile_cbg_path)
arcpy.env.workspace = workspace

#### Load shapefile into a layer and check spatial reference etc:

In [9]:
cbg_layer = shapefile_cbg_path
desc = arcpy.Describe(cbg_layer)
fields = [f.name for f in arcpy.ListFields(cbg_layer)]
count = arcpy.management.GetCount(cbg_layer)
print("Layer cbg_layer info:\n")
print("Shape type:", desc.shapeType)
print("Spatial reference:", desc.spatialReference.name)
print(f"Number of features: {count[0]}\n")
print(f"Field List: {fields}")


Layer cbg_layer info:

Shape type: Polygon
Spatial reference: WGS_1984_Web_Mercator_Auxiliary_Sphere
Number of features: 1057

Field List: ['FID', 'Shape', 'GEOID', 'GEO_ID', 'GEOIDFQ', 'STATEFP', 'COUNTYFP', 'TRACTCE', 'BLKGRPCE', 'NAMELSAD', 'P1_001N', 'trim_acres', 'pop_per_ac', 'pop_sqkm', 'Shape_Leng', 'Shape_Area', 'whole_bg_a', 'whole_bg_s', 'trim_bg_ac', 'trim_bg_sq']
